In [2]:
import numpy as np 
import tensorflow as tf 
import pandas as pd 
import json as js 
import os
import openpyxl as xl 
import itertools as it

In [46]:
def transform_strings(strings_list):

    result_STE_labels = strings_list
    allowed_simbols = "ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮйцукенгшщзхъфывапролджэячсмитьбюQWERTYUIOPASDFGHJKLZXCVBNMqwertyuiopasdfghjklzxcvbnm12345467890"

    for (label_number, STE_label) in enumerate(strings_list):

        tmp_STE_label = ""
        if (type(STE_label) != float):

            for simbol in STE_label:
                        
                if (simbol in allowed_simbols):
                    tmp_STE_label += simbol.lower()

                else:
                    tmp_STE_label += " "
                
            tmp_STE_list = tmp_STE_label.split()
            result_STE_list = []
            iterator = 0
            while iterator < len(tmp_STE_list):

                sub_string = tmp_STE_list[iterator]
                delete_condition = False
                if len(sub_string) >= 4:

                    for simbol in sub_string:

                        if (simbol in "1234567890"):

                            delete_condition = True
                            break

                    if not delete_condition:
                        result_STE_list.append(sub_string)
                
                iterator += 1
            
            result_STE_label = " ".join(sub_string for sub_string in result_STE_list)     
            result_STE_labels[label_number] = result_STE_label
                
        else:
            result_STE_labels[label_number] = STE_label
    
    return result_STE_labels

def sort_strings_list(strings_list):

    result_strings_list = []
    cn_collection = {}
    
    for string in strings_list:
        
        if (type(string) != float):

            tmp_string_list = string.split()
            done_condition = False
            curent_list_part = 1
            while curent_list_part < len(tmp_string_list):

                for sub_string_number in range(len(tmp_string_list) - curent_list_part):

                    if (tmp_string_list[sub_string_number] > tmp_string_list[sub_string_number + 1]):
                        
                        tmp_string_list[sub_string_number], tmp_string_list[sub_string_number + 1] = \
                        tmp_string_list[sub_string_number + 1], tmp_string_list[sub_string_number]

                curent_list_part += 1

            result_string = " ".join(sub_string for sub_string in tmp_string_list)
            result_strings_list.append(result_string)
            cn_collection[result_string] = string
        
        else:
            result_strings_list.append(string)
            cn_collection[string] = string
    
    return (result_strings_list, cn_collection)

Консолидация данных

In [47]:


def collect_data_turnover(data_path):
    
    indexator = 1
    general_STE_labels = []
    data_collection = {
        "Оборотная ведомость сч. 105": {
            "квартал_1":{},
            "квартал_2": {},
            "квартал_3": {},
            "квартал_4": {}
        },
        "Оборотно-Сальдовая ведомость сч. 21": {
            "квартал_1":{},
            "квартал_2": {},
            "квартал_3": {},
            "квартал_4": {}
        },
        "Оборотно-Сальдовая ведомость сч. 101": {
            "квартал_1":{},
            "квартал_2": {},
            "квартал_3": {},
            "квартал_4": {}
        },
    }

    general_STE_labels = []
    for data_batch_f in os.listdir(data_path):
        
        if indexator > 4:
        
            general_STE_labels = []
            indexator = 1

        curent_batch_f = os.path.join(data_path, data_batch_f)
        curent_dataframe = pd.read_excel(curent_batch_f)

        if "сч. 105" in data_batch_f:

            report_type = "Оборотная ведомость " + "сч. 105"
            STE_labels = curent_dataframe["Наименование нефинансового актива"].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            curent_dataframe["Наименование нефинансового актива"] = STE_labels
            curent_dataframe.columns = ["№ п/п", "Инвентарный номенклатурный номер", "Код справочника", "Наименование нефинансового актива",
                                       "Единица измерения", "Остаток на 1 января 2022 г.|дебет|кол-во", 
                                       "Остаток на 1 января 2022 г.|дебет|сумма", "Оборот за 1 квартал 2022 г.|дебет|кол-во", 
                                       "Оборот за 1 квартал 2022 г.|дебет|сумма", "Оборот за 1 квартал 2022 г.|кредит|кол-во", 
                                       "Оборот за 1 квартал 2022 г.|кредит|сумма", "Остаток на 31 марта 2022 г.|дебет|кол-во", 
                                       "Остаток на 31 марта 2022 г.|дебет|сумма"]
        
        elif ("сч. 101" in data_batch_f) or ("сч. 21"  in data_batch_f):

            report_type = "Оборотно-Сальдовая ведомость " + data_batch_f[data_batch_f.find("сч"): data_batch_f.find("за") - 1]
            STE_labels_index = curent_dataframe.columns[0]
            STE_labels = curent_dataframe[STE_labels_index].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            curent_dataframe[STE_labels_index] = STE_labels

        general_STE_labels += STE_labels
        
        data_collection[report_type]["general_STE_labels"] = general_STE_labels
        data_collection[report_type][f"квартал_{indexator}"] = {
            "data": curent_dataframe,
            "STE_labels": STE_labels
        }
        general_STE_labels += STE_labels

        indexator += 1

    return data_collection

data_path = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Обороты по счету"
data_collection = collect_data_turnover(data_path=data_path)
print(data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"].keys())

print(len(data_collection["Оборотная ведомость сч. 105"]["квартал_2"]["STE_labels"]))
print(len(data_collection["Оборотная ведомость сч. 105"]["квартал_3"]["STE_labels"]))
print(len(data_collection["Оборотная ведомость сч. 105"]["квартал_4"]["STE_labels"]))
# data_collection["Оборотная ведомость сч. 105"]["квартал_1"]["data"].iloc[:100].tail()
# data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"]["data"].iloc[:50].tail()
# data_collection["Оборотно-Сальдовая ведомость сч. 21"]["квартал_1"]["data"].iloc[:100].tail()





dict_keys(['data', 'STE_labels'])
[nan, nan, nan, nan, nan, 'клей момент столяр экспресс', 'classicfixкаучуковый tytanprofessional клей монтажный прозрачный', 'professional tytan древесины клей', 'крепежный уголок', nan, 'ботинки внутренними защитными кожаные металлическими носками стандарт утепленные', 'unix держатель предфильтров', 'азов брюки воздействий защиты костюм куртка механических мужской', 'азов брюки воздействий защиты костюм куртка механических мужской', 'азов брюки воздействий защиты костюм куртка механических мужской', 'защиты зима куртка мужская пониженных температур', 'excellence supravision uvex линза открытые очки покрытие прозрачная феос', 'взаимодействий защиты механических перчатки покрытие сити трикотажные', 'воды защиты мужской нефтепродуктов нефти плащ полосой циклон', 'воды защиты мужской нефтепродуктов нефти плащ полосой циклон', 'атлетик внутр композит лайт микрофибра мужские носок полуботинки', 'атлетик внутр композит лайт микрофибра мужские носок полуботин

In [24]:
reminder_data_path = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Складские остатки"
def collect_data_reminder(data_path):

    data_collection = {
        "Ведомость остатков (сч. 105)": {
            "31.03.2022": {},
            "30.06.2022": {},
            "30.09.2022": {},
            "31.12.2022": {}
        },
        "Ведомость остатков (сч. 101)": {
            "31.03.2022": {},
            "30.06.2022": {},
            "30.09.2022": {},
            "31.12.2022": {}
        },
        "Ведомость остатков (сч. 21)": {
            "31.03.2022": {},
            "30.06.2022": {},
            "30.09.2022": {},
            "31.12.2022": {}
        },
    }

    for reminder_sheet_f in os.listdir(data_path):
        
        curent_sample_f = os.path.join(data_path, reminder_sheet_f)
        curent_sample_df = pd.read_excel(curent_sample_f)

        if ("г." not in reminder_sheet_f):
            curent_sample_date = reminder_sheet_f[reminder_sheet_f.find("3"): reminder_sheet_f.find("(") - 1]

        else:
            curent_sample_date = reminder_sheet_f[reminder_sheet_f.find("3"): reminder_sheet_f.find("г.")]

        if "(сч. 105)" in reminder_sheet_f:
            
            report_type = "Ведомость остатков " + reminder_sheet_f[reminder_sheet_f.find("(сч."): reminder_sheet_f.find(")") + 1]
            STE_labels = curent_sample_df["МОЛ"].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            curent_sample_df["МОЛ"] = STE_labels

            # del curent_sample_df["МОЛ"]
            # curent_sample_df.index = STE_labels
        
        elif ("(сч. 21)" in reminder_sheet_f) or ("(сч. 101)" in reminder_sheet_f):
            
            
            report_type = "Ведомость остатков " + reminder_sheet_f[reminder_sheet_f.find("(сч."): reminder_sheet_f.find(")") + 1]
            "Ведомость остатков " + reminder_sheet_f[reminder_sheet_f.find("(сч."): reminder_sheet_f.find(")") + 1]

            for column in curent_sample_df.columns[3:-4]:
                
                del curent_sample_df[column]
            
            del curent_sample_df[curent_sample_df.columns[0]]
            del curent_sample_df[curent_sample_df.columns[0]]

            STE_labels = curent_sample_df[curent_sample_df.columns[0]].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            
            curent_sample_df[curent_sample_df.columns[0]] = STE_labels
            #del curent_sample_df[curent_sample_df.columns[0]]
            curent_sample_df.columns = ["Наименование нефинансового актива", "Балансовая стоимость", "Количество", "Сумма амортизации", "Остаточная стоимость"]
        
        data_collection[report_type][curent_sample_date] = {
            "data": curent_sample_df,
            "STE_labels": STE_labels
        }
    
    return data_collection


reminder_data_collection = collect_data_reminder(data_path=reminder_data_path)
            

30.06.2022
30.06.2022
30.06.2022
30.09.2022
30.09.2022
30.09.2022
31.03.2022
31.03.2022
31.03.2022
31.12.2022
31.12.2022
31.12.2022


In [25]:
print(reminder_data_collection.keys())
print(reminder_data_collection["Ведомость остатков (сч. 105)"].keys())  
print(reminder_data_collection["Ведомость остатков (сч. 21)"].keys())  
print(reminder_data_collection["Ведомость остатков (сч. 101)"].keys())
reminder_data_collection["Ведомость остатков (сч. 105)"]["30.06.2022"]["data"].iloc[:100].tail() 
# reminder_data_collection["Ведомость остатков (сч. 101)"]["30.06.2022"]["data"].iloc[:100].tail() 
# reminder_data_collection["Ведомость остатков (сч. 21)"]["30.06.2022"]["data"].iloc[:100].tail() 

dict_keys(['Ведомость остатков (сч. 105)', 'Ведомость остатков (сч. 101)', 'Ведомость остатков (сч. 21)'])
dict_keys(['31.03.2022', '30.06.2022', '30.09.2022', '31.12.2022'])
dict_keys(['31.03.2022', '30.06.2022', '30.09.2022', '31.12.2022'])
dict_keys(['31.03.2022', '30.06.2022', '30.09.2022', '31.12.2022'])


,МОЛ,Цена,Количество,Сумма
95,torsion для насадка шт шуруповерта,19.04,16.0,304.64
96,daikin nord кондиционер настенный пульт шт,267641.64,1.0,267641.64
97,retractable stenley кровельный лезвием нож с т...,452.08,1.0,452.08
98,sparta в отвертка тестер шт,259.35,2.0,518.70
99,монтажной очиститель пены шт,208.75,2.0,417.50


In [26]:
data_exemple_105 = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Складские остатки\\Ведомость остатков на 30.09.2022 (сч. 105).xlsx"
data_example_101 = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Складские остатки\\Ведомость остатков на 30.09.2022г. (сч. 21).xlsx"
data_example_21 = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Складские остатки\\Ведомость остатков на 30.09.2022г. (сч. 21).xlsx"

data_example_105_df = pd.read_excel(data_exemple_105)
data_example_101_df = pd.read_excel(data_example_101)
data_example_21_df = pd.read_excel(data_example_21)



In [7]:
print(data_example_101_df.columns)
all_columns = data_example_101_df.columns

for columns_name in data_example_101_df.columns[3:-4]:

    del data_example_101_df[columns_name]
del data_example_101_df[all_columns[0]]
del data_example_101_df[all_columns[1]]
data_example_101_df.iloc[:50].tail()

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22'],
      dtype='object')


,Unnamed: 2,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
45,Огнетушитель порошковый ОП-5 (з) (АВСЕ),600,1,0,600
46,Огнетушитель порошковый ОП-5 (з) (АВСЕ),600,1,0,600
47,Огнетушитель порошковый ОП-5 (з) (АВСЕ),600,1,0,600
48,Огнетушитель порошковый ОП-5 (з) (АВСЕ),600,1,0,600
49,Огнетушитель порошковый ОП-5 (з) (АВСЕ),600,1,0,600


In [27]:
trade_inf_ph = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Выгрузка контрактов по Заказчику.xlsx"
cll_path = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\КПГЗ ,СПГЗ, СТЕ.xlsx"
trade_inf_dataframe = pd.read_excel(trade_inf_ph)
cll_dataframe = pd.read_excel(cll_path)

trade_inf_dataframe.tail()

,ID СПГЗ,Наименование СПГЗ,Реестровый номер в РК,Номер лота в закупке,ИКЗ,Заказчик,Наименование (предмет) ГК,Способ определения поставщика,Основание заключения контракта с ед. поставщиком,Статус контракта,...,Дата регистрации,Дата последнего изменения,Срок исполнения с,Срок исполнения по,Дата окончания срока действия,Принадлежность поставщика к МСП на момент заключения ГК,Наименование субъекта РФ поставщика,Закон-основание (44/223),Электронное исполнение,Исполнено поставщиком
3694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3696,3281628.0,"Стенка мебельная, ЛДСП",1449299,1.0,NaN,Закачик 1,"Поставка шкафа для нужд ГБУ ""Инфогород""",NaN,NaN,Исполнен,...,29.01.2018,20.02.2018,30.01.2018,19.02.2018,31.03.2018,Нет,NaN,44.0,Нет,NaN
3697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3698,3364361.0,Услуги по обеспечению мониторинга предоставлен...,5813950,1.0,NaN,Закачик 1,Оказание услуг по обработке данных Комплексной...,электронный конкурс,NaN,Исполнен,...,23.11.2022,21.12.2023,22.11.2022,31.12.2024,31.12.2024,Да,NaN,44.0,Да,2.863343e+08


In [28]:
cll_dataframe.iloc[:100].tail()

,Название СТЕ,наименование характеристик,реф.цена,конечную категорию справочника,КПГЗ код,КПГЗ,СПГЗ код,СПГЗ,Реестровый номер в РК
95,Изготовление рулонных штор LVT 45,Виды;Материал;Плотность ткани;Пропитка;Размер;...,NaN,Изготовление изделий из комбинации различных м...,02.17.01.99,ИЗГОТОВЛЕНИЕ ИЗДЕЛИЙ ИЗ КОМБИНАЦИИ РАЗЛИЧНЫХ М...,NaN,NaN,2827026
96,Стеллаж СКУ,Гарантийный срок;Глубина;Количество полок;Комп...,NaN,Стеллажи складские металлические,01.16.16.02.01,СТЕЛЛАЖИ СКЛАДСКИЕ МЕТАЛЛИЧЕСКИЕ,NaN,NaN,2830507
97,"Доска магнитно-маркерная BRAUBERG стандарт, 10...",Вес;Комплектация;Высота доски;Материал рамы;Ти...,8158.96,Доски магнитно-маркерные,01.15.05.03.01.01,ДОСКИ МАГНИТНО-МАРКЕРНЫЕ,NaN,NaN,2832953
98,"Доска магнитно-маркерная BRAUBERG PREMIUM, 2-с...",Вес;Вращается в горизонтальной плоскости на 36...,26181.79,Доски магнитно-маркерные,01.15.05.03.01.01,ДОСКИ МАГНИТНО-МАРКЕРНЫЕ,NaN,NaN,2832953
99,"Доска магнитно-маркерная BRAUBERG PREMIUM, 2-с...",Тип доски;Высота доски;Материал рамы;Тип крепл...,33251.98,Доски магнитно-маркерные,01.15.05.03.01.01,ДОСКИ МАГНИТНО-МАРКЕРНЫЕ,NaN,NaN,2832953


In [10]:
def transform_single_string(string):

    result_string = ""
    allowed_simbols = "ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮйцукенгшщзхъфывапролджэячсмитьбюQWERTYUIOPASDFGHJKLZXCVBNMqwertyuiopasdfghjklzxcvbnm12345467890"

    for simbol in string:

        if (simbol in allowed_simbols):
            result_string += simbol.lower()
        
        else:
            result_string += " "
    
    return result_string
        
    

In [44]:
STE_collection = []
cn_collection = {}

for ts_number in range(trade_inf_dataframe.shape[0]):

    trade_sample = trade_inf_dataframe.iloc[ts_number]
    cll_dataframe_batch = cll_dataframe[(cll_dataframe["Реестровый номер в РК"] == trade_sample["Реестровый номер в РК"])
                                    & (cll_dataframe["КПГЗ код"] == trade_sample["Конечный код КПГЗ"])
                                    & (cll_dataframe["КПГЗ"] == trade_sample["Конечное наименование КПГЗ"])]
    
    trade_sample_STE = cll_dataframe_batch["Название СТЕ"].to_list()
    trade_sample_STE = transform_strings(trade_sample_STE)
    trade_sample_STE, sub_cn_collection = sort_strings_list(trade_sample_STE)
    

    if len(trade_sample_STE) > 2:

        for variant in trade_sample_STE:

            STE_collection.append((variant, 
                                   trade_sample["Конечное наименование КПГЗ"], 
                                   sub_cn_collection[variant]))

    elif len(trade_sample_STE) == 1:
        STE_collection.append((trade_sample_STE[0], 
                               trade_sample["Конечное наименование КПГЗ"], 
                               sub_cn_collection[trade_sample_STE[0]]))
    


print(STE_collection)

[('оказание охране помещений услуг', 'УСЛУГИ ЧАСТНОЙ ОХРАНЫ (ВЫСТАВЛЕНИЕ ПОСТА ОХРАНЫ)', 'оказание услуг охране помещений'), ('охрана физическая', 'УСЛУГИ ЧАСТНОЙ ОХРАНЫ (ВЫСТАВЛЕНИЕ ПОСТА ОХРАНЫ)', 'физическая охрана'), ('охрана физическая', 'УСЛУГИ ЧАСТНОЙ ОХРАНЫ (ВЫСТАВЛЕНИЕ ПОСТА ОХРАНЫ)', 'физическая охрана'), ('officespace белый блок бокс записи пластиковый', 'БЛОКИ БУМАЖНЫЕ ДЛЯ ЗАПИСЕЙ', 'блок записи officespace пластиковый бокс белый'), ('staff блокнот вида гребень жесткая клетка подложка', 'БЛОКНОТЫ', 'блокнот гребень жесткая подложка клетка staff вида'), ('stenography блокнот картон клетка обложка офсет спираль', 'БЛОКНОТЫ', 'блокнот stenography спираль офсет клетка обложка картон'), ('color бумага пачка цветная', 'БУМАГА ДЛЯ ОФИСНОЙ ТЕХНИКИ', 'бумага цветная color пачка'), ('berlingo губка досок европодвес магните', 'ГУБКИ ДЛЯ МАРКЕРНЫХ ДОСОК', 'губка досок berlingo магните европодвес'), ('белый дозатор жидкого мерида мыла', 'ИНВЕНТАРЬ ДЛЯ САНУЗЛОВ ПЛАСТМАССОВЫЙ', 'дозатор ж

In [30]:
need_dataframe = data_collection["Оборотная ведомость сч. 105"]["квартал_1"]["data"]
found_con = 0
for STE_sample in STE_collection:

    for data_batch_key in data_collection.keys():
        
        curent_data_batch = data_collection[data_batch_key]
        for time_kvartal in curent_data_batch:

            if time_kvartal != "general_STE_labels":
                
                data_frame = curent_data_batch[time_kvartal]["data"]
                
                if (data_batch_key == "Оборотная ведомость сч. 105"):
                    serched_dataframe = data_frame[data_frame["Наименование нефинансового актива"] == STE_sample[0]]
                
                else:
                    serched_dataframe = data_frame[data_frame[data_frame.columns[0]] == STE_sample[0]]


                if (serched_dataframe.shape[0] != 0):
                    found_con += 1

print(found_con)

318


In [37]:
found_reg = {}
found_con = 0

indexator = 0
for trade_sample in STE_collection:

    STE_0 = data_collection["Оборотная ведомость сч. 105"]["general_STE_labels"]
    STE_1 = data_collection["Оборотно-Сальдовая ведомость сч. 101"]["general_STE_labels"]
    STE_2 = data_collection["Оборотно-Сальдовая ведомость сч. 21"]["general_STE_labels"]

    data_labels = ["Оборотная ведомость сч. 105", "Оборотно-Сальдовая ведомость сч. 101", "Оборотно-Сальдовая ведомость сч. 21"]


    for (sample_number, (label_0, label_1, label_2)) in enumerate(zip(STE_0, STE_1, STE_2)):
        
        labels = [label_0, label_1, label_2]
        done_condition = False
        if done_condition:
            break

        for (label_number, label) in enumerate(labels):

            if type(label) != float:
                
                in_label = all([ts_string in label.split() for ts_string in trade_sample[0].split()
                                if (type(label) != float) & (type(trade_sample[0]) != float)])
                if (in_label):
                    
                    found_con += 1
                    serched_dataframe = data_collection[data_labels[label_number]][]
                    found_reg[trade_sample[0]] = {
                        "тип_ведомости": data_labels[label_number],
                        "расположение_в_ведомости": sample_number,
                        "СТЕ наименование": trade_sample[2],
                        "КПГЗ": trade_sample[1]
                        }
                    done_condition = True
                    break
        


with open("C:\\Users\\1\\Desktop\\LDTProject\\chek_results.json", "w") as json_file:
    js.dump(found_reg, json_file)

print(found_reg)
print(found_con)

{'stenography блокнот картон клетка л обложка офсет спираль': {'тип_ведомости': 'Оборотная ведомость сч. 105', 'расположение_в_ведомости': 1536, 'СТЕ наименование': 'блокнот л stenography спираль офсет клетка обложка картон', 'КПГЗ': 'БЛОКНОТЫ'}, 'matrix набор надфилей шт': {'тип_ведомости': 'Оборотная ведомость сч. 105', 'расположение_в_ведомости': 1609, 'СТЕ наименование': 'набор надфилей matrix шт', 'КПГЗ': 'ИНСТРУМЕНТ РУЧНОЙ ПРОЧИЙ'}, 'postfix белый конверт лента силиконовая упак шт': {'тип_ведомости': 'Оборотная ведомость сч. 105', 'расположение_в_ведомости': 267, 'СТЕ наименование': 'конверт postfix белый силиконовая лента шт упак', 'КПГЗ': 'КОНВЕРТЫ ПОЧТОВЫЕ'}, 'ботинки внутренними защитными кожаные м металлическими носками р с стандарт утепленные': {'тип_ведомости': 'Оборотная ведомость сч. 105', 'расположение_в_ведомости': 1481, 'СТЕ наименование': 'ботинки кожаные стандарт м утепленные с металлическими внутренними защитными носками р', 'КПГЗ': 'ОБУВЬ С ЗАЩИТНЫМ МЕТАЛЛИЧЕСКИМ 

In [ ]:
found_reg = {}
found_con = 0

indexator = 0
for trade_sample in STE_collection:

    data_batch_0 = data_collection["Оборотная ведомость сч. 105"]
    data_batch_1 = data_collection["Оборотно-Сальдовая ведомость сч. 101"]
    data_batch_2 = data_collection["Оборотно-Сальдовая ведомость сч. 21"]
    
    for kvartal in data_batch_0.keys():

        if kvartal != "general_STE_labels":

            for (STE_label_number, STE_label) in enumerate(data_batch_0[kvartal]["STE_labels"]):

                if type(STE_label) != float:

                    in_label = all([ts_sub_string in STE_label.split() 
                                    for ts_sub_string in trade_sample[0] 
                                    if type(trade_sample[0]) != float])
                    if in_label:
                        
                        found_reg[trade_sample[2]] = {
                            "тим ведомости": "Оборотная ведомость сч. 105",
                            "КПГЗ": trade_sample[1],
                            f"{kvartal}": {
                                "position in dataframe": STE_label_number
                            }  
                        }
                        break

            for (STE_label_number, STE_label) in enumerate(data_batch_1[kvartal]["STE_labels"]):

                if type(STE_label) != float:

                    in_label = all([ts_sub_string in STE_label.split() 
                                    for ts_sub_string in trade_sample[0] 
                                    if type(trade_sample[0]) != float])
                    if in_label:
                        
                        found_reg[trade_sample[2]] = {
                            "тим ведомости": "Оборотно-Сальдовая ведомость сч. 101",
                            "КПГЗ": trade_sample[1],
                            f"{kvartal}": {
                                "position in dataframe": STE_label_number
                            }  
                        }
                        break

            for (STE_label_number, STE_label) in enumerate(data_batch_2[kvartal]["STE_labels"]):

                if type(STE_label) != float:

                    in_label = all([ts_sub_string in STE_label.split() 
                                    for ts_sub_string in trade_sample[0] 
                                    if type(trade_sample[0]) != float])
                    if in_label:
                        
                        found_reg[trade_sample[2]] = {
                            "тим ведомости": "Оборотно-Сальдовая ведомость сч. 21",
                            "КПГЗ": trade_sample[1],
                            f"{kvartal}": {
                                "position in dataframe": STE_label_number
                            }  
                        }
                        break


            
    
        


with open("C:\\Users\\1\\Desktop\\LDTProject\\chek_results.json", "w") as json_file:
    js.dump(found_reg, json_file)

print(found_reg)
print(found_con)

In [45]:
found_reg_test = {}

found_con_test = 0
found_uncon_test = 0

found_con_rem_test = 0
found_uncon_rem_test = 0

time_classification = {
    "квартал_1": "31.03.2022",
    "квартал_2": "30.06.2022",
    "квартал_3": "30.09.2022",
    "квартал_4": "31.12.2022"
}

batch_classification = {
    "Оборотная ведомость сч. 105": "Ведомость остатков (сч. 105)",
    "Оборотно-Сальдовая ведомость сч. 101": "Ведомость остатков (сч. 101)",
    "Оборотно-Сальдовая ведомость сч. 21": "Ведомость остатков (сч. 21)"
}

for trade_sample in STE_collection:

    
    for data_batch in data_collection.keys():

        reminder_report_type = batch_classification[data_batch]
        curent_data_batch = data_collection[data_batch]
        curent_data_batch_rem = reminder_data_collection[reminder_report_type]

        for time_kvartal in curent_data_batch.keys():
            
            if (data_batch == "Оборотная ведомость сч. 105") and (reminder_report_type == "Ведомость остатков (сч. 105)"):
                
                if time_kvartal != "general_STE_labels":
                    
                    amount_reg = {}
                    reminder_time_kvartal = time_classification[time_kvartal]

                    kvartal_dataframe = curent_data_batch[time_kvartal]["data"]
                    kvartal_dataframe_reminder = curent_data_batch_rem[reminder_time_kvartal]["data"]
                    
                    serched_dataframe = kvartal_dataframe[kvartal_dataframe["Наименование нефинансового актива"] == trade_sample[0]]
                    serched_dataframe_rem = kvartal_dataframe_reminder[kvartal_dataframe_reminder["МОЛ"] == trade_sample[0]]

                    desired_condition = False
                    if serched_dataframe.shape[0] != 0:
                        
                        desired_condition = True
                        found_con_test += 1
                        start_amount = serched_dataframe["Остаток на 1 января 2022 г.|дебет|сумма"].to_numpy()[0]
                        if np.isnan(start_amount):
                            start_amount = serched_dataframe["Оборот за 1 квартал 2022 г.|дебет|сумма"].to_numpy()[0]
                        

                        amount_reg["начальная стоимость"] = start_amount
                        
                            
                    if serched_dataframe_rem.shape[0] != 0:
                        
                        desired_condition = True
                        found_con_rem_test += 1
                        end_amount = serched_dataframe_rem["Сумма"].to_numpy()[0]
                        if np.isnan(end_amount):
                            end_amount = serched_dataframe["Остаток на 31 марта 2022 г.|дебет|сумма"].to_numpy()[0]  
                        
                        if np.isnan(end_amount) == False:
                            amount_reg["конечная стоимость"] = end_amount
                        
                        
                    elif serched_dataframe.shape[0] != 0:

                        desired_condition = True
                        found_uncon_rem_test += 1
                        end_amount = serched_dataframe["Остаток на 31 марта 2022 г.|дебет|сумма"].to_numpy()[0]
                    
                        if np.isnan(end_amount) == False:
                            amount_reg["конечная стоимость"] = end_amount

                    if desired_condition:
                        
                        found_reg_test[trade_sample[2]] = {}
                        found_reg_test[trade_sample[2]]["Классификация"] = trade_sample[1]
                        found_reg_test[trade_sample[2]][time_kvartal] = amount_reg

print(found_con_test)
print(found_con_rem_test)



with open("C:\\Users\\1\\Desktop\\LDTProject\\test_results.json", "w") as json_file:
    js.dump(found_reg_test, json_file)

239
22
{'блокнот stenography спираль офсет клетка обложка картон': {'Классификация': 'БЛОКНОТЫ', 'квартал_4': {'начальная стоимость': 7870}}, 'набор надфилей matrix': {'Классификация': 'ИНСТРУМЕНТ РУЧНОЙ ПРОЧИЙ', 'квартал_4': {'начальная стоимость': 566.15, 'конечная стоимость': 566.15}}, 'конверт postfix белый силиконовая лента упак': {'Классификация': 'КОНВЕРТЫ ПОЧТОВЫЕ', 'квартал_1': {'начальная стоимость': 17114.2}}, 'мастика опечатывания': {'Классификация': 'МАСТИКИ ГИДРОИЗОЛЯЦИОННЫЕ', 'квартал_2': {'начальная стоимость': 1209.6}}, 'ботинки кожаные стандарт утепленные металлическими внутренними защитными носками': {'Классификация': 'ОБУВЬ С ЗАЩИТНЫМ МЕТАЛЛИЧЕСКИМ ПОДНОСКОМ', 'квартал_4': {'начальная стоимость': 2115.46}}, 'фильтр противогазовый unix': {'Классификация': 'РЕСПИРАТОРЫ', 'квартал_4': {'начальная стоимость': 13440, 'конечная стоимость': 13440}}, 'полумаска unix размер': {'Классификация': 'РЕСПИРАТОРЫ', 'квартал_4': {'начальная стоимость': 2020, 'конечная стоимость': 20

In [ ]:
found_reg = {}
found_con = 0

STE_test = data_collection["Оборотная ведомость сч. 105"]["general_STE_labels"]

def sort_strings_list(strings_list):

    result_strings_list = []
    for string in strings_list:
        
        if (type(string) != float):

            tmp_string_list = string.split()
            done_condition = False
            curent_list_part = 1
            while curent_list_part < len(tmp_string_list):

                for sub_string_number in range(len(tmp_string_list) - curent_list_part):

                    if (tmp_string_list[sub_string_number] > tmp_string_list[sub_string_number + 1]):
                        
                        tmp_string_list[sub_string_number], tmp_string_list[sub_string_number + 1] = \
                        tmp_string_list[sub_string_number + 1], tmp_string_list[sub_string_number]

                curent_list_part += 1

            result_string = " ".join(sub_string for sub_string in tmp_string_list)
            result_strings_list.append(result_string)
        
        else:
            result_strings_list.append(string)
    
    return result_strings_list

result_STE_labels = sort_strings_list(STE_test[:])
print(len(result_STE_labels))

        



[nan, nan, nan, nan, nan, '750г клей момент столяр экспресс', '310мл classicfixкаучуковый tytanprofessional клей монтажный прозрачный', '33d3 professional tytan wb белый750г для древесины клей пва', '50х50х35х2 крепежный уголок', nan, '43 ботинки внутренними защитными кожаные м металлическими носками пу р с стандарт тпу утепленные', 'unix держатель предфильтров', '100 170 176 96 азов брюки воздействий для защиты и костюм куртка механических мужской опз от р', '100 182 188 96 азов брюки воздействий для защиты и костюм куртка механических мужской опз от р', '104 108 182 188 азов брюки воздействий для защиты и костюм куртка механических мужской опз от р', '104 108 182 188 для защиты зима куртка мужская от пониженных р температур', '9192370 excellence supravision uvex ван линза открытые очки покрытие прозрачная феос', '9 взаимодействий для защиты и механических опз от пвх перчатки покрытие р рук сити трикотажные', '104 108 182 188 воды для защиты мужской нефтепродуктов нефти от плащ полосо

In [39]:
found_con = 0
for true_STE_label in found_reg.keys():

    true_STE = found_reg[true_STE_label]
    curent_data_batch = data_collection[true_STE["тип_ведомости"]]
    done_condition = False
    total_kvartals = {}

    for time_k in curent_data_batch.keys():
        
        if (time_k != "general_STE_labels"):
            
            dataframe = curent_data_batch[time_k]["data"]
            if (true_STE["тип_ведомости"] == "Оборотная ведомость сч. 105"):
                        
                for ts_number in range(dataframe.shape[0]):
                            
                    curent_ds = dataframe.iloc[ts_number]
                    if (ts_number > 1) and type(curent_ds["Наименование нефинансового актива"]) != float:

                        in_label = all([tsl_string in curent_ds["Наименование нефинансового актива"].split() for tsl_string in true_STE_label.split()])
                        if (in_label):
                                    
                            found_con += 1
                            start_cost = curent_ds["Остаток на 1 января 2022 г.|дебет|сумма"]
                            if (np.isnan(start_cost)):
                                start_cost = curent_ds["Оборот за 1 квартал 2022 г.|дебет|сумма"]
                            
                            curent_k_report = {
                                "start_cost": start_cost
                            }
                            total_kvartals[time_k] = curent_k_report
  
                            break

                        else:

                            start_cost = None
            

            else:
                start_cost = None

            

    
    found_reg[true_STE_label]["кварталы"] = total_kvartals
        

print(found_con, len(found_reg))
with open("C:\\Users\\1\\Desktop\\LDTProject\\chek_results.json", "w") as json_file:
    js.dump(found_reg, json_file)







154 92


In [34]:
found_con = 0
time_classification = {
    "квартал_1": "31.03.2022",
    "квартал_2": "30.06.2022",
    "квартал_3": "30.09.2022",
    "квартал_4": "31.12.2022"
}

batch_classification = {
    "Оборотная ведомость сч. 105": "Ведомость остатков (сч. 105)",
    "Оборотно-Сальдовая ведомость сч. 101": "Ведомость остатков (сч. 101)",
    "Оборотно-Сальдовая ведомость сч. 21": "Ведомость остатков (сч. 21)"
}
inv_time_classification = {value: key for (value, key) in zip(time_classification.values(), time_classification.keys())}

for trade_sample in found_reg.keys():

    report_type = batch_classification[found_reg[trade_sample]["тип_ведомости"]]
    curent_batch = reminder_data_collection[report_type]
    for time_kvartal in curent_batch.keys():

        inv_time_kvartal = inv_time_classification[time_kvartal]
        curent_dataframe = curent_batch[time_kvartal]["data"]

        for ts_number in range(curent_dataframe.shape[0]):
                
            curent_ds = curent_dataframe.iloc[ts_number]
            if (report_type == "Ведомость остатков (сч. 105)"):

                if (type(curent_ds["МОЛ"]) != float):

                    in_labels = all([ts_string in curent_ds["МОЛ"].split() for ts_string in trade_sample.split()])
                    if (in_labels):
                            
                        found_con += 1
                        end_cost = curent_ds["Сумма"]
                        found_reg[trade_sample]["кварталы"][inv_time_kvartal]["end_cost"] = end_cost
                        break
                    
                    else:
                        
                        batch_type = found_reg[trade_sample]["тип_ведомости"]
                        ts_position = found_reg[trade_sample]["расположение_в_ведомости"]
                        print(ts_position, data_collection[batch_type][inv_time_kvartal]["data"].shape)
                        end_cost = data_collection[batch_type][inv_time_kvartal]["data"].iloc[ts_position]
                        print(end_cost)
                        found_reg[trade_sample]["кварталы"][inv_time_kvartal]["end_cost"] = end_cost

print(found_con, len(found_reg))
with open("C:\\Users\\1\\Desktop\\LDTProject\\chek_results.json", "w") as json_file:
    js.dump(found_reg, json_file)
                    
                        
                        


                            

                        
        
    

1536 (177, 13)


IndexError: single positional indexer is out-of-bounds